In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
%run parse.ipynb

In [ ]:
# alright, let's try with one movement, one performer!
X = torch.Tensor(parseInputFile('../bach-violin-dataset/dataset/source-input/emil-telmanyi/bwv1002/emil-telmanyi_bwv1002_mov1.csv'))
y = torch.Tensor(parseGroundtruthFile('../bach-violin-dataset/dataset/ground-truth/emil-telmanyi/bwv1002/emil-telmanyi_bwv1002_mov1.csv'))

In [ ]:
print(X)
print(y)
print(X.shape)
print(y.shape)

In [ ]:
# Starting with linear regression. For this, we only need one linear layer.
# For now, we don't need to build our own module yet because linear regression is simple. However, for an MLP, we will need multiple layers.
model = nn.Linear(3, 2)

In [ ]:
# it was retaining nan values for its parameters...
model.reset_parameters()

In [ ]:
# Define our loss function (mean squared error) to be used in the grad descent step
loss = nn.MSELoss()

# Performs the gradient descent steps
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)

In [ ]:
# Template code from https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial2/Introduction_to_PyTorch.html#The-Basics-of-PyTorch
def train_model(model, optimizer, input_notes, truth, loss_module, num_epochs=100):
    # Set model to train mode
    model.train()

    # Training loop
    for epoch in range(num_epochs):

        ## Step 1: Move input data to device (only strictly necessary if we use GPU)
        # data_inputs = data_inputs.to(device)
        # data_labels = data_labels.to(device)

        ## Step 2: Run the model on the input data
        preds = model(input_notes)
        print(preds)
        print(model.weight)
        print(model.bias)

        ## Step 3: Calculate the loss
        loss = loss_module(preds, y)
        print(loss)

        ## Step 4: Perform backpropagation
        # Before calculating the gradients, we need to ensure that they are all zero.
        # The gradients would not be overwritten, but actually added to the existing ones.
        optimizer.zero_grad()
        # Perform backpropagation
        loss.backward()

        ## Step 5: Update the parameters
        optimizer.step()

In [ ]:
train_model(model, optimizer, X, y, loss)

In [ ]:
X_test = torch.Tensor(parseInputFile('../bach-violin-dataset/dataset/source-input/emil-telmanyi/bwv1001/emil-telmanyi_bwv1001_mov3.csv'))
y_test = torch.Tensor.tolist(model(X_test))

print(model.weight)

X_test = torch.Tensor.tolist(X_test)
print(X_test)
for i in range(len(y_test)):
    y_test[i].append(X_test[i][2])

print(y_test)